# 1. Chuẩn bị model và config:

In [55]:
import html
import re

URL_RE = re.compile(r"https?://\S+|www\.\S+")
USER_RE = re.compile(r"@(\w+)")
HASHTAG_RE = re.compile(r"#(\w+)")

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
!unzip "/content/drive/MyDrive/NLP/Final_Project/Demo/BertModel.zip" -d /content/bert_model

Archive:  /content/drive/MyDrive/NLP/Final_Project/Demo/BertModel.zip
replace /content/bert_model/outputs_bert/bert_best/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [58]:
!pip install -q transformers torch gradio

In [59]:
MODEL_DIR = "/content/bert_model/outputs_bert/bert_best"


In [60]:
# Load tokenize & model:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_DIR
)

model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [61]:
id2label = {
    0: "Hate Speech",
    1: "Offensive",
    2: "Neither"
}

label2id = {v: k for k, v in id2label.items()}

model.config.id2label = id2label
model.config.label2id = label2id

In [62]:
# Label mapping:
label_map = {
    0: "Hate Speech",
    1: "Offensive",
    2: "Neither"
}

In [63]:
# Kiểm tra label mapping:
print(model.config.id2label)


{0: 'Hate Speech', 1: 'Offensive', 2: 'Neither'}


In [64]:
# Mapping màu sắc:
LABEL_COLORS = {
    "Hate Speech": "#d32f2f",   # đỏ đậm
    "Offensive": "#f57c00",     # cam
    "Neither": "#388e3c"        # xanh lá
}

In [65]:
def clean_text(text):
    text = str(text)
    text = html.unescape(text)

    # Thay vì xóa sạch, dùng token để giữ tín hiệu "có URL/mention"
    text = URL_RE.sub(' [URL] ', text)
    text = USER_RE.sub(' [USER] ', text)

    # Giữ nội dung hashtag, chỉ bỏ dấu #
    text = HASHTAG_RE.sub(r' \1 ', text)

    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    # Chỉ bật nếu bạn dùng model uncased (vd: bert-base-uncased)
    text = text.lower()

    return text

# 2. Hàm dự đoán của mô hình:

In [66]:
# Hàm dự đoán:
def predict_hate_speech(text):
    if text.strip() == "":
        return "Please enter a sentence.", ""

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)

    conf, pred_id = torch.max(probs, dim=1)

    label = model.config.id2label[pred_id.item()]
    confidence = conf.item()

    return label, f"{confidence:.4f}"

In [67]:
# !pip install captum

# 3. Tích hợp cơ chế giải thích bằng phân bố attention:

In [68]:
# Tích hợp cơ chế giải thích bằng cách highlight các từ chú ý:
import torch
import numpy as np
from captum.attr import IntegratedGradients

# Explainability: Integrated Gradients (word/token highlight)
def _merge_wordpieces(tokens, scores):
    # Merge BERT wordpieces (##) into whole words by summing scores.
    words = []
    word_scores = []
    cur = ""
    cur_score = 0.0

    for t, s in zip(tokens, scores):
        if t in ["[CLS]", "[SEP]", "[PAD]"]:
            continue
        if t.startswith("##"):
            cur += t[2:]
            cur_score += float(s)
        else:
            if cur:
                words.append(cur)
                word_scores.append(cur_score)
            cur = t
            cur_score = float(s)
    if cur:
        words.append(cur)
        word_scores.append(cur_score)
    return words, np.array(word_scores, dtype=np.float32)

def explain_ig(text, target_label=None, max_length=128, n_steps=50):
    model.eval()

    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_attention_mask=True,
    )
    input_ids = enc["input_ids"].to(device)
    attn_mask = enc["attention_mask"].to(device)

    def forward_embeds(embeds):
        out = model(inputs_embeds=embeds, attention_mask=attn_mask)
        return out.logits

    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attn_mask).logits
        probs = torch.softmax(logits, dim=1)[0].detach().cpu().numpy()
        pred_id = int(probs.argmax())
        pred_label = id2label[pred_id]

    if target_label is None:
        target = pred_id
    else:
        if isinstance(target_label, str):
            inv = {v: k for k, v in id2label.items()}
            target = inv[target_label]
        else:
            target = int(target_label)

    embeddings = model.get_input_embeddings()
    input_embeds = embeddings(input_ids)
    baseline_ids = torch.full_like(input_ids, tokenizer.pad_token_id)
    baseline_embeds = embeddings(baseline_ids)

    ig = IntegratedGradients(forward_embeds)
    attributions = ig.attribute(
        inputs=input_embeds,
        baselines=baseline_embeds,
        target=target,
        n_steps=n_steps
    )

    token_scores = attributions.sum(dim=-1).squeeze(0).detach().cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0).detach().cpu().tolist())

    words, word_scores = _merge_wordpieces(tokens, token_scores)

    imp = np.abs(word_scores)
    if imp.max() > 0:
        imp = imp / imp.max()
    return pred_id, pred_label, probs, words, imp

def render_highlight_html(words, scores, max_words=80):
    words = words[:max_words]
    scores = scores[:max_words]
    spans = []
    for w, s in zip(words, scores):
        s = float(max(0.0, min(1.0, s)))
        spans.append(
            f'<span style="background: rgba(255, 0, 0, {0.15 + 0.75*s}); padding:2px 4px; margin:1px; border-radius:4px; display:inline-block;">{w}</span>'
        )
    return "<div style='line-height: 2.0;'>" + " ".join(spans) + "</div>"

def render_confidence_html(probs, id2label):
    probs = np.array(probs)
    best_idx = probs.argmax()

    rows = []
    for i, p in enumerate(probs):
        label = id2label[i]
        color = LABEL_COLORS.get(label, "#000000")

        style = ""
        if i == best_idx:
            style = f"font-weight:bold; background-color: {color}20;"

        rows.append(
            f"""
            <tr style="{style}">
                <td style="padding:6px 12px; color:{color};">
                    {label}
                </td>
                <td style="padding:6px 12px;">
                    {p:.4f}
                </td>
            </tr>
            """
        )

    html = f"""
    <table style="border-collapse: collapse; margin-top: 8px; width: 60%;">
        <tr>
            <th style="text-align:left; padding:6px 12px; border-bottom:1px solid #ccc;">
                Label
            </th>
            <th style="text-align:left; padding:6px 12px; border-bottom:1px solid #ccc;">
                Confidence
            </th>
        </tr>
        {''.join(rows)}
    </table>
    """
    return html

# 4. Thiết lập UI cho Demo web:

## 4.1. Vẽ bar-chart cho phân bố xác suất nhãn:

In [69]:
# Vẽ bar chart cho phân bố xác suất nhãn:
import matplotlib.pyplot as plt

def plot_confidence_bar(probs, id2label):
    labels = [id2label[i] for i in range(len(probs))]
    colors = [LABEL_COLORS.get(l, "#999999") for l in labels]

    fig, ax = plt.subplots(figsize=(6, 3))
    ax.bar(labels, probs, color=colors)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Confidence")
    ax.set_title("Confidence Distribution")

    for i, v in enumerate(probs):
        ax.text(i, v + 0.02, f"{v:.2f}", ha="center", fontsize=9)

    plt.tight_layout()
    return fig

## 4.2. Tích hợp CSS để có UI đẹp:

In [70]:
# Tích hợp CSS để có UI demo xịn:
CUSTOM_CSS = """
#predict-btn {
    font-size: 18px;
    font-weight: bold;
    padding: 10px;
}

.label-box textarea {
    font-size: 20px !important;
    font-weight: bold !important;
    text-align: center;
}

.conf-box textarea {
    font-size: 16px !important;
    text-align: center;
}

.section-title {
    font-size: 22px;
    font-weight: 700;
    margin-bottom: 8px;
}

.sub-title {
    font-size: 16px;
    font-weight: 600;
    color: #555;
}
"""

In [71]:
# Hàm predict với cơ chế attention:
def gradio_predict_with_explain(text):
    if text.strip() == "":
        return (
            "N/A",
            "0.0000",
            "<i>No confidence available</i>",
            None,
            "<i>No explanation</i>"
        )

    # Apply cleaning to the input text
    cleaned_text = clean_text(text)

    pred_id, pred_label, probs, words, imp = explain_ig(cleaned_text)

    pred_conf = float(probs[pred_id])

    confidence_html = render_confidence_html(probs, id2label)
    bar_plot = plot_confidence_bar(probs, id2label)
    highlight_html = render_highlight_html(words, imp)

    return (
        pred_label,
        f"{pred_conf:.4f}",
        confidence_html,
        bar_plot,
        highlight_html
    )

## 4.3. Giao diện Demo với hai cột chính:

In [72]:
import gradio as gr

with gr.Blocks(css=CUSTOM_CSS, title="Hate Speech Detection Demo") as demo:

    gr.Markdown(
        """
        <div class="section-title">🧠 Hate Speech Detection using BERT</div>
        <div class="sub-title">
        Explainability with Integrated Gradients (Token-level Highlighting)
        </div>
        """
    )

    with gr.Row():

        # ================= LEFT COLUMN =================
        with gr.Column(scale=1):

            input_text = gr.Textbox(
                lines=4,
                placeholder="Enter an English sentence...",
                label="Input Text"
            )

            predict_btn = gr.Button(
                "🚀 Predict",
                elem_id="predict-btn"
            )

            output_label = gr.Textbox(
                label="Predicted Label",
                elem_classes="label-box"
            )

            output_conf = gr.Textbox(
                label="Predicted Label Confidence",
                elem_classes="conf-box"
            )

        # ================= RIGHT COLUMN =================
        with gr.Column(scale=1.3):

            gr.Markdown("<div class='section-title'>📊 Model Confidence</div>")

            output_all_conf = gr.HTML(label="Confidence for All Labels")

            output_bar = gr.Plot(label="Confidence Bar Chart")

            gr.Markdown("<div class='section-title'>🔍 Explainability</div>")

            output_explain = gr.HTML(label="Token Importance Visualization")

    predict_btn.click(
        fn=gradio_predict_with_explain,
        inputs=input_text,
        outputs=[
            output_label,
            output_conf,
            output_all_conf,
            output_bar,
            output_explain
        ]
    )

demo.launch(debug=True)

/tmp/ipython-input-3285868730.py:3: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=CUSTOM_CSS, title="Hate Speech Detection Demo") as demo:
/usr/local/lib/python3.12/dist-packages/gradio/layouts/column.py:59: UserWarning: 'scale' value should be an integer. Using 1.3 will cause issues.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ccca34f4f84bf43b4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ccca34f4f84bf43b4a.gradio.live
